# Overhead


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

Step 1: Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

ratings_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ratings.csv")
users_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/users.csv")
movies_df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/movies.tsv", sep = "\t")

'/content'

Step 2: Prepare Data for KNN Algorithm

In [ ]:
from scipy.sparse import csr_matrix #AELX(11/18): is this ever used?

#convert the ratings dataframe into a sparse matrix. Rows = movieIDs and columns = userIDs
user_ratings_df = ratings_df.pivot(index='movieID', columns='userID', values='rating').fillna(0)

# Build the model


In [ ]:
#import sci kit learn's nearest neighbor class
from sklearn.neighbors import NearestNeighbors

#generate model with selected hyperparameters
#Hyperparameters: 
#metric: cosine similarity instead of Euclidean Distance due to sparsity of data matrix
#Algorithm: brute force (most computatonally expensive, but also the most accurate. Could use ball tree to only search specified area for neighbor)
#n_neighbors = 20. Can try a few different values of neighbors to speed up or increase accuracy
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)

#fit the model with the data matrix
knn.fit(user_ratings_df)

NearestNeighbors(algorithm='brute', metric='cosine')

Step 4: create a function that generates n best movie recomendations

In [ ]:
def knn_calculator(movieIDs):
  #create a list that stores the top three recommendations from each movie in movies vector
  all_raw_recommendations = []

  #print title of movie that user would like to get recomendations for
  print('Recomendation engine starting to generate recomendations')

  for movieID in movieIDs:

    #calculate distances and indices for the top closest n movies to the movie input to function
    distances, indices = knn.kneighbors([user_ratings_df.iloc[movieID]])
    print('distances', distances)
    print('indices:', indices)

    #get a list of raw recomendations
    raw_recommends = \
              sorted(
                  list(
                      zip(
                          indices.squeeze().tolist(),
                          distances.squeeze().tolist()
                      )
                  ),
                  key=lambda x: x[1]
              )[:0:-1]
    
    #collect all the distances for all movies at these movies
    all_raw_recommendations.extend(raw_recommends)
  
  raw_recommendations_df = pd.DataFrame(all_raw_recommendations, columns=('movieID', 'distance'))
  #take the average distance when multiple movies are returned.
  recommendations_df = raw_recommendations_df.groupby('movieID').mean('distance').sort_values('distance', ascending=False)
  
  #this returns all recommendations, but must be trimmed to remove the ones the user ahs already seen in the next step.
  return recommendations_df 

In [ ]:
#movies is a list of movie names in string format
movies = [2, 586, 250] #['Jumanji', 'Home Alone', 'Star Wars: Episode IV - A New Hope']
recommendations_df = knn_calculator(movies)
print(recommendations_df)

Recomendation engine starting to generate recomendations
distances [[8.88178420e-16 5.02967115e-01 5.30149871e-01 5.30233882e-01
  5.31376848e-01]]
indices: [[  2   1 726 175 504]]
distances [[1.55431223e-15 6.47106927e-01 6.58472177e-01 6.94958835e-01
  6.95125830e-01]]
indices: [[ 586  149  691  874 1081]]
distances [[0.         0.53821402 0.56581163 0.56740954 0.57005538]]
indices: [[ 250 1226  543  249 1343]]
         distance
movieID          
1081     0.695126
874      0.694959
691      0.658472
149      0.647107
1343     0.570055
249      0.567410
543      0.565812
1226     0.538214
504      0.531377
175      0.530234
726      0.530150
1        0.502967


# Viewer Objecct